In [1]:
import PyPDF2
import nltk
from nltk.tokenize import word_tokenize
import re
import pandas as pd
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
import os

In [2]:
# Funcion para convertir el contenido de un pdf a txt
def getPDFFileContentToTXT(pdfFile):
    myPDFFile = PyPDF2.PdfFileReader(pdfFile)
    
#Creamos un archivo txt con la informacion del pdf 
    with open('pdfContenido.txt', 'w') as pdf_output:
        for page in range (myPDFFile.getNumPages()):
            data = myPDFFile.getPage(page).extractText()
            pdf_output.write(data)

# Leemos la infromacion que acabamos de escribir e quitamos los espacios de lineas
    with open('pdfContenido.txt', 'r') as myPDFContent:
        return myPDFContent.read().replace('\n',' ')

## Programa para analizar y calificar CV´s por Andres Saldaña

### Requerimientos de Vacantes

In [3]:
df = pd.read_csv('Requisitos_Vacantes.csv')

In [4]:
df = df.drop("Otros",axis = 1)

In [5]:
df = df.fillna(" ") #rellenamos las casillas sin nada con un espacio

In [6]:
df

,Igualador,Facturacion
0,Igualación de colores,Facturación de clientes
1,Captura de bitácoras,Cobro de ventas mostrador
2,Trabajo en equipo,Elaboración de corte diario
3,Pintura automotriz,Elaboración de Pedidos
4,,Encargado de facturación


In [7]:
#Que vacante quieres cargar?

vacante = "Facturacion"
df[vacante]

0        Facturación de clientes
1      Cobro de ventas mostrador
2    Elaboración de corte diario
3         Elaboración de Pedidos
4       Encargado de facturación
Name: Facturacion, dtype: object

### Hacer una lista con los requerimientos de la vacante 

In [8]:
requerimientos = ""

for i in df[vacante]:
    requerimientos = requerimientos +i + " "
    
print(requerimientos)

Facturación de clientes Cobro de ventas mostrador Elaboración de corte diario Elaboración de Pedidos Encargado de facturación 


### Convertir CV a TXT

In [9]:
pdfFileContent = getPDFFileContentToTXT('CVF1.pdf')
os.remove("pdfContenido.txt") #removemos el archivo de texto creado (opcional)

In [10]:
pdfFileContent

'Perla Leticia Sandoval EscobedoMonterrey, N. L.perlaleticiasandovalescob2_z7p@indeedemail.com+52 81 1798 1253Dispuesto a trasladarse a: cualquier parteExperiencia laboralAuxiliar AdministrativoCentro Regional de Acumuladores SA de CV.  -  Monterrey, N. L. junio 2015 - Actualmente Atención a clientes ,cajera , auxiliar de facturación, créditos y cobranza ,manejo de efectivo , cheque , transferencias , conciliaciones de saldo , reporte de caja , arqueos, cierres al día , depósitos bancarios, elaboración de notas de crédito ,notas de cargo ,aplicación de pagos, recibo de efectivo ,habilidades matemáticas ,manejo de problemas y conflictos , manejo de estrés ,manejo de correo ,portales de clientes ,conmutador, facilidad de palabra, facturas a revisión, trato con los vendedores ,encargada de departamento de atención a clientes y servicio a domicilio . ENCARGADA DE CREDITO Y COBRANZA /FACTURACION. Papeleria y Libreria Patria de Monterrey SA de CV  -  Monterrey, N. L. junio 2010 - mayo 2015 R

### Separamos las palabras pegadas TXT

In [11]:
count = 1

for letter in range(len(pdfFileContent)): #funcion para limpiar y corregir errores de transformacion a txt
    
    count +=1
    if count == len(pdfFileContent):
        break
    
    l1 = pdfFileContent[letter]
    l2 = pdfFileContent[letter + 1]
    l3 = pdfFileContent[letter + 2]
    
    if l1.islower() == True and l2.isupper() == True and l3.islower() == True: #Errores de \n
        print("Errores por separar (letras): " + l1,l2,l3)
        pdfFileContent = pdfFileContent.replace(l1+l2+l3,l1+" "+l2+l3)
        
    if l1.isdigit() == True and l2.isupper() == True and l3.islower() == True: #Numeros pegados a letras mayusculas 
        print("Errores por separar (numero): " + l1,l2,l3)
        pdfFileContent = pdfFileContent.replace(l1+l2+l3,l1+" "+l2+l3)

Errores por separar (letras): o M o
Errores por separar (numero): 3 D i
Errores por separar (letras): e E x
Errores por separar (letras): l A u
Errores por separar (letras): o C e


### Tokenize y removemos caracteres innecesarios (Tokens 1 y 2)

In [12]:
Tokens_1 = word_tokenize(pdfFileContent) 
#Hacemos token el CV de la persona y los requrimientos de las vacnates
Tokens_vacantes_1 = word_tokenize(requerimientos)

In [13]:
punctuation = re.compile(r'[-+/.?!,:;()•@[0-9]') #lista de caracteres que removeremos

Tokens_2 = []
for words in Tokens_1:
    word = punctuation.sub("",words)
    if len(word)>0:
        Tokens_2.append(word)
        
Tokens_vacantes_2 = []
for words in Tokens_vacantes_1:
    word = punctuation.sub("",words)
    if len(word)>0:
        Tokens_vacantes_2.append(word)

In [14]:
Tokens_2[10:15] #tokens con puras palabras sin caracteres

['trasladarse', 'a', 'cualquier', 'parte', 'Experiencia']

In [15]:
Tokens_vacantes_2[0:5]

['Facturación', 'de', 'clientes', 'Cobro', 'de']

### Stem y remove stop words de el CV y los Requerimientos (Tokens 3 y 4)

In [16]:
stopw = stopwords.words("spanish") #Removemos las palabras de relleno que no aportan informacion
Tokens_3 = [word for word in Tokens_2 if word not in stopw] 
Tokens_vacantes_3 = [word for word in Tokens_vacantes_2 if word not in stopw] 

In [17]:
Tokens_3[10:15]

['cualquier', 'parte', 'Experiencia', 'laboral', 'Auxiliar']

In [18]:
Tokens_vacantes_3[0:5]

['Facturación', 'clientes', 'Cobro', 'ventas', 'mostrador']

In [19]:
sbst = SnowballStemmer('spanish')

Tokens_4 = []
for words in Tokens_3:
    word = sbst.stem(words)
    Tokens_4.append(word)
    #print(words+": "+ sbst.stem(words))

dic_revertir = {}
Tokens_vacantes_4 = []
for words in Tokens_vacantes_3:
    word = sbst.stem(words)
    Tokens_vacantes_4.append(word)
    dic_revertir[word] = words

In [20]:
Tokens_4[10:15]

['cualqui', 'part', 'experient', 'laboral', 'auxili']

In [21]:
Tokens_vacantes_4[0:5]

['factur', 'client', 'cobr', 'vent', 'mostrador']

In [22]:
sentence_1 = " ".join(Tokens_4)
sentence_1

'perl letici sandoval escobed monterrey n lperlaleticiasandovalescob_zp indeedemailcom dispuest traslad cualqui part experient laboral auxili administr centr regional acumul sa cv monterrey n l juni actual atencion client cajer auxili factur credit cobranz manej efect chequ transferent concili sald report caj arque cierr dia deposit bancari elabor not credit not carg aplic pag recib efect habil matemat manej problem conflict manej estres manej corre portal client conmut facil palabr factur revision trat vendedor encarg departament atencion client servici domicili encarg de credit y cobranz facturacion papeleri libreri patri monterrey sa cv monterrey n l juni may respons cobranz telefon facturacion atencion client trat direct mensajer revision factur cobranz diari conciliacion bancari realiz not credit not carg identificacion aplicacion pag recib efect chequ archiv revision factur logr ten carter sald dia facturacion recient habil matemat manej problem conflict manej estres comunic educ

In [23]:
sentence_2 = " ".join(Tokens_vacantes_4)
sentence_2

'factur client cobr vent mostrador elabor cort diari elabor ped encarg factur'

### Buscar Requerimientos en CV | Bag Of Words

In [24]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

CountVec = CountVectorizer(ngram_range=(1,1)) # to use bigrams ngram_range=(2,2)

#transform
Count_data = CountVec.fit_transform([sentence_1,sentence_2])
 
#create dataframe
cv_dataframe = pd.DataFrame(Count_data.toarray(),columns = CountVec.get_feature_names_out())
cv_dataframe

,actual,acumul,administr,administracion,aplic,aplicacion,archiv,arque,atencion,auxili,...,servici,tecnic,telefon,ten,transferent,traslad,trat,vendedor,vent,word
0,1,1,1,1,1,1,1,1,3,2,...,1,2,1,1,1,1,2,1,0,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [25]:
count = 0
count2 = 0
matches = []

for i,j in zip(cv_dataframe.iloc[0],cv_dataframe.iloc[1]):
    if i >= 1 and j >=1:
        count += 1
        matches.append(cv_dataframe.columns.values[count2])
    count2 += 1

### Dar una Calificacion Basada en Los Requerimientos 

In [26]:
final_score = (count / len(Tokens_vacantes_4))*100

print("El perfil del vacante se alinea " + str(round(final_score)) + "% con el de los requerimeitnos" )

El perfil del vacante se alinea 42% con el de los requerimeitnos


In [27]:
for i in matches: #lo que se encontro dentro del curriculum
    print(dic_revertir[i])

clientes
diario
Elaboración
Encargado
facturación
